- This is a copy of zero_shot_evaluate.ipynb, but run on the BART model that has been fine tuned.
- Will it perform better than no fine tuning?
- The inputs to this notebook are generated by scripts/zero_shot_all_labels_fine_tune.py

In [5]:
from transformers import pipeline
import numpy as np
import pandas as pd
from sklearn import metrics
import torch
import datasets
from datasets import Dataset
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm

import sys  
sys.path.insert(0, '/root/Project-CS224N-ED-Disposition/')
from scripts import utils
import importlib
importlib.reload(utils)

<module 'scripts.utils' from '/root/Project-CS224N-ED-Disposition/scripts/utils.py'>

In [2]:
raw_predictions = datasets.load_from_disk("/root/data/zero_shot_predictions_fine_tune/all")
print(raw_predictions)
# print(raw_predictions[0])

Dataset({
    features: ['sequence', 'labels', 'scores'],
    num_rows: 146149
})


In [6]:
# takes ~1 minute
pdf = utils.zero_shot_results_to_pandas(raw_predictions)
pdf

Loading cached processed dataset at /root/data/zero_shot_predictions_fine_tune/all/cache-9524498990c97f09.arrow


,sequence,Pneumonia,Pneumothorax,Pleural Effusion,Edema,Fracture,Infection,Aspiration,Cardiomegaly,Opacities,...,Small bowel obstruction,Pancreatitis,Splenic laceration,Liver laceration,Colitis,Pyelonephritis,Nephrolithiasis,Malignancy,Pericaridial effusion,Aortic dissection
0,1. Interval increase in perihilar opacities a...,0.135703,0.398748,0.570067,0.842026,0.848332,0.419689,0.673083,0.587732,0.995342,...,0.445921,0.313729,0.822987,0.591597,0.373370,0.572738,0.458242,0.236872,0.855167,0.554643
1,1. No acute intracranial abnormality. 2. Mil...,0.245431,0.602361,0.553857,0.352542,0.509726,0.458867,0.787410,0.373166,0.775706,...,0.509244,0.219510,0.601616,0.421869,0.204419,0.384576,0.278221,0.124652,0.566979,0.616844
2,1. Multiple loops of dilated small bowel with...,0.197257,0.511806,0.663249,0.680145,0.917176,0.488680,0.443935,0.531910,0.810278,...,0.894133,0.158328,0.874023,0.446938,0.240820,0.884780,0.383641,0.815383,0.852387,0.656839
3,1. Interval increased loculated moderate meta...,0.300323,0.785208,0.993042,0.713539,0.823174,0.099582,0.068302,0.668755,0.977929,...,0.523761,0.256485,0.842192,0.550861,0.400135,0.625163,0.549736,0.196960,0.798811,0.653301
4,1. Redemonstrated fracture of the femoral hea...,0.134623,0.168862,0.578540,0.400249,0.871628,0.246647,0.618986,0.267638,0.056146,...,0.200394,0.125356,0.420986,0.255616,0.195666,0.519628,0.336748,0.142691,0.523768,0.234666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146144,1. Acute superior endplate compression deform...,0.181499,0.371067,0.688158,0.609727,0.816371,0.357900,0.582027,0.337661,0.699905,...,0.272966,0.224897,0.742339,0.314153,0.300519,0.382193,0.256639,0.278824,0.769120,0.451844
146145,1. Diffuse anasarca and mesenteric edema. Pat...,0.105284,0.473656,0.951259,0.952216,0.924335,0.255376,0.778405,0.340836,0.942848,...,0.468474,0.123660,0.884163,0.356863,0.205391,0.322606,0.234982,0.099396,0.600775,0.545775
146146,1. No ultrasound evidence of deep venous thro...,0.456509,0.722582,0.960836,0.840743,0.931550,0.652678,0.858157,0.641345,0.900526,...,0.741831,0.514513,0.960681,0.803062,0.472672,0.842929,0.530468,0.516530,0.962089,0.773129
146147,1. Mild fluid overload with new small pleural...,0.009877,0.037124,0.820997,0.133875,0.486811,0.149398,0.198253,0.177307,0.917464,...,0.291761,0.039582,0.723231,0.281979,0.092588,0.078307,0.077726,0.046869,0.381490,0.298664


In [7]:
pdf.to_csv('/root/data/zero_shot_predictions_fine_tune/scores.csv')

In [8]:
chex_df = pd.read_csv('/root/data/chexbert_results.csv')
chex_df.head()

,index,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,No Finding,Report Impression
0,0,NaN,NaN,1.0,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1. Interval increase in perihilar opacities a...
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1. No acute intracranial abnormality. 2. Mil...
2,2,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1. Multiple loops of dilated small bowel with...
3,3,NaN,NaN,NaN,NaN,NaN,1.0,-1.0,-1.0,NaN,1.0,NaN,NaN,NaN,NaN,1. Interval increased loculated moderate meta...
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1. Redemonstrated fracture of the femoral hea...


In [9]:
# results are very similar to zero_shot_results.ipynb, which only evaluated 30k rows
for label in utils.chex_labels:
    utils.evaluate_naive_bayes(pdf, chex_df, label)

Fracture


/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.89      1.00      0.94    130038
         1.0       0.00      0.00      0.00     16111

    accuracy                           0.89    146149
   macro avg       0.44      0.50      0.47    146149
weighted avg       0.79      0.89      0.84    146149

AUC 0.5
Edema


/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.95      1.00      0.98    139224
         1.0       0.00      0.00      0.00      6925

    accuracy                           0.95    146149
   macro avg       0.48      0.50      0.49    146149
weighted avg       0.91      0.95      0.93    146149

AUC 0.5
Cardiomegaly


...

/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98    141417
         1.0       0.00      0.00      0.00      4732

    accuracy                           0.97    146149
   macro avg       0.48      0.50      0.49    146149
weighted avg       0.94      0.97      0.95    146149

AUC 0.5
Pneumothorax


/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00    144829
         1.0       0.00      0.00      0.00      1320

    accuracy                           0.99    146149
   macro avg       0.50      0.50      0.50    146149
weighted avg       0.98      0.99      0.99    146149

AUC 0.5
Pleural Effusion
              precision    recall  f1-score   support

         0.0       0.92      1.00      0.96    134443
         1.0       0.00      0.00      0.00     11706

    accuracy                           0.92    146149
   macro avg       0.46      0.50      0.48    146149
weighted avg       0.85      0.92      0.88    146149

AUC 0.5


/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/envs/l2v/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
labels_df = utils.get_labels(pdf, chex_df)
labels_df

,sequence,Pneumonia,Pneumothorax,Pleural Effusion,Edema,Fracture,Infection,Aspiration,Cardiomegaly,Opacities,...,Small bowel obstruction,Pancreatitis,Splenic laceration,Liver laceration,Colitis,Pyelonephritis,Nephrolithiasis,Malignancy,Pericaridial effusion,Aortic dissection
0,1. Interval increase in perihilar opacities a...,False,False,False,False,False,False,True,False,True,...,False,False,True,True,False,True,False,False,True,True
1,1. No acute intracranial abnormality. 2. Mil...,False,False,False,False,False,False,True,False,True,...,True,False,True,False,False,False,False,False,True,True
2,1. Multiple loops of dilated small bowel with...,False,False,False,False,False,False,False,False,True,...,True,False,True,False,False,True,False,True,True,True
3,1. Interval increased loculated moderate meta...,False,False,False,False,False,False,False,False,True,...,True,False,True,True,False,True,True,False,True,True
4,1. Redemonstrated fracture of the femoral hea...,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,True,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146144,1. Acute superior endplate compression deform...,False,False,False,False,False,False,True,False,True,...,False,False,True,False,False,False,False,False,True,False
146145,1. Diffuse anasarca and mesenteric edema. Pat...,False,False,False,False,False,False,True,False,True,...,False,False,True,False,False,False,False,False,True,True
146146,1. No ultrasound evidence of deep venous thro...,False,False,False,False,False,True,True,False,True,...,True,True,True,True,False,True,True,True,True,True
146147,1. Mild fluid overload with new small pleural...,False,False,False,False,False,False,False,False,True,...,False,False,True,False,False,False,False,False,False,False


In [11]:
labels_df.to_csv('/root/data/zero_shot_predictions/labels_t50.csv')